# Finding Missing Person

### Importing Neccessary Libraries

In [89]:
import numpy as np#used for numerical analysis
#import keras
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


In [90]:
tensorflow.__version__

'2.3.0'

In [91]:
tensorflow.keras.__version__

'2.4.0'

### Image Data Agumentation

In [92]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [93]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory('../dataset/trainset',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='binary')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('../dataset/testset',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='binary') 

Found 174 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [94]:
print(x_train.class_indices)#checking the number of classes

{'Found Missing': 0, 'Normal': 1}


In [95]:
print(x_test.class_indices)#checking the number of classes

{'Found Missing': 0, 'Normal': 1}


In [96]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 80, 1: 94})

### Creating the model

In [97]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid')) # softmax for more than 2



In [98]:
classifier.summary()#summary of our model

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               802944    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

### Compiling the model

In [99]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [100]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Epoch 1/20
35/35 [==============================] - 5s 144ms/step - loss: 0.7237 - accuracy: 0.5402 - val_loss: 0.7224 - val_accuracy: 0.6667
Epoch 2/20
35/35 [==============================] - 5s 139ms/step - loss: 0.6679 - accuracy: 0.5517 - val_loss: 0.8024 - val_accuracy: 0.6667
Epoch 3/20
35/35 [==============================] - 5s 149ms/step - loss: 0.6907 - accuracy: 0.5460 - val_loss: 0.7319 - val_accuracy: 0.6667
Epoch 4/20
35/35 [==============================] - 5s 140ms/step - loss: 0.6630 - accuracy: 0.6264 - val_loss: 0.7534 - val_accuracy: 0.5500
Epoch 5/20
35/35 [==============================] - 5s 143ms/step - loss: 0.6500 - accuracy: 0.5862 - val_loss: 0.7541 - val_accuracy: 0.6667
Epoch 6/20
35/35 [==============================] - 5s 140ms/step - loss: 0.6407 - accuracy: 0.6379 - val_loss: 0.7536 - val_accuracy: 0.6500
Epoch 7/20
35/35 [==============================] - 5s 145ms/step - loss: 0.5672 - accuracy: 0.7069 - val_loss: 0.9637 - val_accuracy: 0.3833
Epoch 

### Saving our model

In [101]:
# Save the model
classifier.save('missing_person.h5')

In [101]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [102]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("missing_person.h5") #loading the model for testing

In [103]:
img = image.load_img(r"D:\SmartBridge\VEC\AI based Natural disaster analysis\dataset\trainset\Found Missing\gettyimages-917897974-612x612.jpg",grayscale=False,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred=model.predict_classes(x)
#y=model.predict(x)
#pred = np.argmax(y,axis=1)#predicting the classes
pred

array([[0]])

In [106]:
index=['Found Missing', 'Normal']
result=str(index[pred[0][0]])
result

'Found Missing'